In [1]:
import numpy as np
import pandas as pd

import matplotlib as mpl
import matplotlib.pyplot as plt

import seaborn as sns
import seaborn.objects as so

from functools import reduce
from itertools import combinations

from scipy import stats

# configure pandas
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)

home = 0

if home:
    dpath = '/Users/cglab/projects/abcd/data/abcd5.1-rser/'
else:
    dpath = '/home/cglab/projects/abcd/data/abcd5.1-rser/'

#### Function to pull deriviatives

In [2]:
def get_deriviatives(df, table_file, table_key, merge_how):
    dat = pd.read_csv(dpath + table_file)
    # get table name, which is the string before the period
    table = table_file.split('.')[0]
    # get column names sub and event which will need for merging dataframes
    # it's the same for each df so overwriting is fine
    se_nms = dat.columns[:2].values.tolist()
    # deriviative variables desired
    deriviative_cols = table_key[table_key['Table']==table]['Variable'].values.tolist()
    deriviative_cols += se_nms
    # merge with overall with INNER join bc we dont want to exclude participants who have task mri data but not resting or vice versa
    print('Prior to merge rs df size is {0} and other df shape is {1}'.format(df.shape, dat[deriviative_cols].shape))
    if 'src_subject_id' not in df.columns:
        # for first table assign it to df
        df = dat[deriviative_cols].copy()
    else:
        # all others are merged
        df = df.merge(dat[deriviative_cols], how=merge_how, on=['src_subject_id', 'eventname'])
    print('Any duplicated columns? {}'.format(df.columns.duplicated().any()))
    print('New rs df size is {}'.format(df.shape))
    return df

### Load Area deprivation index

In [3]:
led = pd.read_csv(dpath + 'led_l_adi.csv')
led.head()

,src_subject_id,eventname,reshist_addr1_adi_edu_l,reshist_addr1_adi_edu_h,reshist_addr1_adi_work_c,reshist_addr1_adi_income,reshist_addr1_adi_in_dis,reshist_addr1_adi_home_v,reshist_addr1_adi_rent,reshist_addr1_adi_mortg,reshist_addr1_adi_home_o,reshist_addr1_adi_crowd,reshist_addr1_adi_unemp,reshist_addr1_adi_pov,reshist_addr1_adi_b138,reshist_addr1_adi_sp,reshist_addr1_adi_ncar,reshist_addr1_adi_ntel,reshist_addr1_adi_nplumb,reshist_addr1_adi_wsum,reshist_addr1_adi_perc,reshist_addr2_adi_edu_l,reshist_addr2_adi_edu_h,reshist_addr2_adi_work_c,reshist_addr2_adi_income,reshist_addr2_adi_in_dis,reshist_addr2_adi_home_v,reshist_addr2_adi_rent,reshist_addr2_adi_mortg,reshist_addr2_adi_home_o,reshist_addr2_adi_crowd,reshist_addr2_adi_unemp,reshist_addr2_adi_pov,reshist_addr2_adi_b138,reshist_addr2_adi_sp,reshist_addr2_adi_ncar,reshist_addr2_adi_ntel,reshist_addr2_adi_nplumb,reshist_addr2_adi_wsum,reshist_addr2_adi_perc,reshist_addr3_adi_edu_l,reshist_addr3_adi_edu_h,reshist_addr3_adi_work_c,reshist_addr3_adi_income,reshist_addr3_adi_in_dis,reshist_addr3_adi_home_v,reshist_addr3_adi_rent,reshist_addr3_adi_mortg,reshist_addr3_adi_home_o,reshist_addr3_adi_crowd,reshist_addr3_adi_unemp,reshist_addr3_adi_pov,reshist_addr3_adi_b138,reshist_addr3_adi_sp,reshist_addr3_adi_ncar,reshist_addr3_adi_ntel,reshist_addr3_adi_nplumb,reshist_addr3_adi_wsum,reshist_addr3_adi_perc
0,NDAR_INV005V6D2C,baseline_year_1_arm_1,4.861931,87.37314,95.01950,45609.0,2.946340,271100.0,995.0,1804.0,16.290290,2.440678,9.991899,17.770597,27.568094,29.079160,14.237288,1.966102,0.000000,97.757630,32.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NDAR_INV007W6H7B,baseline_year_1_arm_1,3.559711,94.81628,99.60899,129961.0,3.567694,897800.0,1605.0,1534.0,26.772322,2.930622,6.254295,3.665339,33.268700,0.000000,78.110050,4.814593,0.751274,27.144264,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NDAR_INV00BD7VDC,baseline_year_1_arm_1,0.635838,96.58959,97.06238,84150.0,1.344857,149600.0,845.0,1165.0,73.894310,0.000000,3.420132,1.204819,6.977778,5.863454,7.986447,0.000000,0.000000,104.137800,45.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NDAR_INV00CY2MDM,baseline_year_1_arm_1,2.196885,93.04783,92.36878,63977.0,2.714429,108100.0,713.0,1028.0,61.082424,2.690397,7.476038,6.062932,12.870530,15.349195,13.741722,1.200331,0.000000,110.804726,66.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NDAR_INV00NPMHND,baseline_year_1_arm_1,0.612392,94.52450,94.13694,81602.0,0.538996,272400.0,1100.0,1463.0,83.935550,3.215434,3.634927,3.376623,5.488851,13.073593,0.578778,0.964630,0.000000,92.811060,25.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
led.eventname.value_counts()

eventname
baseline_year_1_arm_1    11215
Name: count, dtype: int64

In [5]:
pullvars = pd.read_excel(dpath + 'ContextualAdversity.xlsx')
pullvars.head()

,Variable,Table,Measure,Factor
0,reshist_addr1_adi_unemp,led_l_adi,Area Deprivation Index - unemployment,Harsh SES
1,reshist_addr1_adi_edu_l,led_l_adi,Area Deprivation Index - education,Harsh SES
2,reshist_addr1_adi_crowd,led_l_adi,Area Deprivation Index - crowding,Harsh SES
3,reshist_addr1_adi_pov,led_l_adi,Area Deprivation Index - poverty,Harsh SES
4,reshist_addr1_opat_kfrpp_avg,led_l_socmob,Social Mobility,Harsh SES


### Load, Select, & Merge data, 'abcd_p_demo.csv'
* Get relevant deriviatives from abcd_p_demo.csv
* and merge with overall adv rf

In [6]:
adv = pd.DataFrame()
adv = get_deriviatives(adv, 'abcd_p_demo.csv', pullvars, 'left')

Prior to merge rs df size is (0, 0) and other df shape is (48807, 4)
Any duplicated columns? False
New rs df size is (48807, 4)


In [7]:
adv.eventname.value_counts()

eventname
baseline_year_1_arm_1       11868
1_year_follow_up_y_arm_1    11220
2_year_follow_up_y_arm_1    10908
3_year_follow_up_y_arm_1    10123
4_year_follow_up_y_arm_1     4688
Name: count, dtype: int64

### Load, Select, & Merge data, 'led_l_adi.csv'
* Get relevant deriviatives from led_l_adi.csv
* and merge with overall adv rf

In [8]:
adv = get_deriviatives(adv, 'led_l_adi.csv', pullvars, 'left')

Prior to merge rs df size is (48807, 4) and other df shape is (11215, 6)
Any duplicated columns? False
New rs df size is (48807, 8)


In [9]:
adv.eventname.value_counts()

eventname
baseline_year_1_arm_1       11868
1_year_follow_up_y_arm_1    11220
2_year_follow_up_y_arm_1    10908
3_year_follow_up_y_arm_1    10123
4_year_follow_up_y_arm_1     4688
Name: count, dtype: int64

### Load, Select, & Merge data, 'led_l_socmob.csv'
* Get relevant deriviatives from led_l_socmob.csv
* and merge with overall adv rf

In [10]:
adv = get_deriviatives(adv, 'led_l_socmob.csv', pullvars, 'left')

Prior to merge rs df size is (48807, 8) and other df shape is (10780, 3)
Any duplicated columns? False
New rs df size is (48807, 9)


In [11]:
adv.eventname.value_counts()

eventname
baseline_year_1_arm_1       11868
1_year_follow_up_y_arm_1    11220
2_year_follow_up_y_arm_1    10908
3_year_follow_up_y_arm_1    10123
4_year_follow_up_y_arm_1     4688
Name: count, dtype: int64

### Load, Select, & Merge data, 'ce_p_nsc.csv'
* Get relevant deriviatives from cd_p_nsc.csv
* and merge with overall adv rf

In [12]:
adv = get_deriviatives(adv, 'ce_p_nsc.csv', pullvars, 'left')

Prior to merge rs df size is (48807, 9) and other df shape is (38815, 5)
Any duplicated columns? False
New rs df size is (48807, 12)


In [13]:
data = pd.read_csv(dpath + 'abcd_p_demo.csv')
data.loc[data['src_subject_id'].isin(adv['src_subject_id']), ['demo_prnt_ed_v2', 'demo_prtnr_ed_v2', 'src_subject_id']]

,demo_prnt_ed_v2,demo_prtnr_ed_v2,src_subject_id
0,13.0,13.0,NDAR_INV003RTV85
1,NaN,NaN,NDAR_INV003RTV85
2,NaN,NaN,NDAR_INV003RTV85
3,NaN,NaN,NDAR_INV003RTV85
4,6.0,999.0,NDAR_INV005V6D2C
...,...,...,...
48802,17.0,13.0,NDAR_INVZZZP87KR
48803,NaN,NaN,NDAR_INVZZZP87KR
48804,NaN,NaN,NDAR_INVZZZP87KR
48805,NaN,NaN,NDAR_INVZZZP87KR


In [17]:
yr1_vars = [v for v in adv.columns if 'adi' in v or 'opat_kfrpp' in v or 'neigh' in v]
yr1_vars += ['src_subject_id']
adv.loc[adv['eventname']=='1_year_follow_up_y_arm_1', yr1_vars]

,reshist_addr1_adi_unemp,reshist_addr1_adi_edu_l,reshist_addr1_adi_crowd,reshist_addr1_adi_pov,reshist_addr1_opat_kfrpp_avg,neighborhood1r_p,neighborhood2r_p,neighborhood3r_p,src_subject_id
1,NaN,NaN,NaN,NaN,NaN,5.0,5.0,4.0,NDAR_INV003RTV85
5,NaN,NaN,NaN,NaN,NaN,1.0,4.0,3.0,NDAR_INV005V6D2C
12,NaN,NaN,NaN,NaN,NaN,5.0,5.0,4.0,NDAR_INV00BD7VDC
15,NaN,NaN,NaN,NaN,NaN,2.0,3.0,2.0,NDAR_INV00CY2MDM
20,NaN,NaN,NaN,NaN,NaN,4.0,2.0,3.0,NDAR_INV00HEV6HB
...,...,...,...,...,...,...,...,...,...
48781,NaN,NaN,NaN,NaN,NaN,4.0,4.0,4.0,NDAR_INVZZLZCKAY
48786,NaN,NaN,NaN,NaN,NaN,5.0,5.0,4.0,NDAR_INVZZNX6W2P
48791,NaN,NaN,NaN,NaN,NaN,5.0,4.0,4.0,NDAR_INVZZPKBDAC
48796,NaN,NaN,NaN,NaN,NaN,5.0,5.0,4.0,NDAR_INVZZZ2ALR6


In [16]:
data.head()

,src_subject_id,eventname,demoi_p_select_language___1,demo_prim,demo_brthdat_v2,demo_ed_v2,demo_adopt_agex_v2,demo_adopt_agex_v2_bl_dk,demo_sex_v2,demo_gender_id_v2,demo_race_a_p___10,demo_race_a_p___11,demo_race_a_p___12,demo_race_a_p___13,demo_race_a_p___14,demo_race_a_p___15,demo_race_a_p___16,demo_race_a_p___17,demo_race_a_p___18,demo_race_a_p___19,demo_race_a_p___20,demo_race_a_p___21,demo_race_a_p___22,demo_race_a_p___23,demo_race_a_p___24,demo_race_a_p___25,demo_race_a_p___77,demo_race_a_p___99,demo_ethn_v2,demo_ethn2_v2,demo_origin_v2,demo_years_us_v2,demo_years_us_v2_dk,demo_relig_v2,demo_prnt_age_v2,demo_prnt_age_v2_bl_refuse,demo_prnt_gender_id_v2,demo_prnt_race_a_v2___10,demo_prnt_race_a_v2___11,demo_prnt_race_a_v2___12,demo_prnt_race_a_v2___13,demo_prnt_race_a_v2___14,demo_prnt_race_a_v2___15,demo_prnt_race_a_v2___16,demo_prnt_race_a_v2___17,demo_prnt_race_a_v2___18,demo_prnt_race_a_v2___19,demo_prnt_race_a_v2___20,demo_prnt_race_a_v2___21,demo_prnt_race_a_v2___22,demo_prnt_race_a_v2___23,demo_prnt_race_a_v2___24,demo_prnt_race_a_v2___25,demo_prnt_race_a_v2___77,demo_prnt_race_a_v2___99,naas_id,naas_mom_id,naas_id_dad,naas_birthplace,naas_raised,naas_comm_contact,naas_pride,naas_self_rating,naas_traditions,demo_prnt_ethn_v2,demo_prnt_ethn2_v2,demo_prnt_16,demo_prnt_16a,demo_prnt_origin_v2,demo_biofather_v2,demo_biomother_v2,demo_matgrandm_v2,demo_matgrandf_v2,demo_patgrandm_v2,demo_patgrandf_v2,demo_prnt_years_us_v2,demo_prnt_years_us_v2_dk,demo_prnt_marital_v2,demo_prnt_ed_v2,demo_prnt_empl_v2,demo_prnt_empl_time,demo_prnt_income_v2,demo_prnt_prtnr_v2,demo_prnt_prtnr_bio,demo_prnt_prtnr_adopt,demo_prtnr_ed_v2,demo_prtnr_empl_v2,demo_prtnr_empl_time,demo_prtnr_income_v2,demo_comb_income_v2,demo_fam_exp1_v2,demo_fam_exp2_v2,demo_fam_exp3_v2,demo_fam_exp4_v2,demo_fam_exp5_v2,demo_fam_exp6_v2,demo_fam_exp7_v2,demo_roster_v2,demo_roster_v2_refuse,fam_roster_2c_v2,...,demo_prnt_prtnr_adopt_l,demo_prtnr_ed_v2_l,demo_prtnr_empl_v2_l,demo_prtnr_empl_time_l,demo_prtnr_indust_refuse_l,demo_prtnr_income_v2_l,demo_child_time_v2_l,demo_child_time2_v2_l,demo_child_time2_v2_dk_l,demo_child_time3_v2_l,demo_comb_income_v2_l,demo_roster_v2_l,demo_roster_v2_refuse_l,fam_roster_2c_v2_l,fam_roster_3c_v2_l,fam_roster_4c_v2_l,fam_roster_5c_v2_l,fam_roster_6c_v2_l,fam_roster_7c_v2_l,fam_roster_8c_v2_l,fam_roster_9c_v2_l,fam_roster_10c_v2_l,fam_roster_11c_v2_l,fam_roster_12c_v2_l,fam_roster_13c_v2_l,fam_roster_14c_v2_l,fam_roster_15c_v2_l,demo_fam_exp1_v2_l,demo_fam_exp2_v2_l,demo_fam_exp3_v2_l,demo_fam_exp4_v2_l,demo_fam_exp5_v2_l,demo_fam_exp6_v2_l,demo_fam_exp7_v2_l,demo_yrs_1_l,demo_yrs_2_l,demo_yrs_2a_l,demo_yrs_2b_l,demo_yrs_2_no_display_l___1,demo_med_insur_f_p,demo_med_insur_g_p,demo_med_insur_h_p,demo_prnt_race_acs_p__10,demo_prnt_race_acs_p__11,demo_prnt_race_acs_p__12,demo_prnt_race_acs_p__13,demo_prnt_race_acs_p__14,demo_prnt_race_acs_p__15,demo_prnt_race_acs_p__16,demo_prnt_race_acs_p__17,demo_prnt_race_acs_p__18,demo_prnt_race_acs_p__19,demo_prnt_race_acs_p__20,demo_prnt_race_acs_p__21,demo_prnt_race_acs_p__22,demo_prnt_race_acs_p__23,demo_prnt_race_acs_p__24,demo_prnt_race_acs_p__777,demo_prnt_race_acs_p__999,demo_prnt_ed_v2_2yr_l,demo_prnt_ethnic_acs_p,demo_prtnr_ed_v2_2yr_l,demo_med_insur_a_p,demo_med_insur_b_p,demo_med_insur_c_p,demo_med_insur_d_p,demo_med_insur_e_p,demo_nat_lang_3_yrs_eng_p___7,demo_nat_lang_3_yrs_eng_p___8,demo_nat_lang_3_yrs_eng_p___9,demo_nat_lang_3_yrs_eng_p___10,demo_nat_lang_3_yrs_eng_p___11,demo_nat_lang_3_yrs_eng_p___12,demo_nat_lang_3_yrs_eng_p___13,demo_nat_lang_3_yrs_eng_p___14,demo_nat_lang_3_yrs_other_p__0,demo_nat_lang_3_yrs_other_p__1,demo_nat_lang_3_yrs_other_p__2,demo_nat_lang_3_yrs_other_p__3,demo_nat_lang_3_yrs_other_p__4,demo_nat_lang_3_yrs_other_p__5,demo_nat_lang_3_yrs_other_p__6,demo_nat_lang_3_yrs_other_p__7,demo_nat_lang_3_yrs_other_p__8,demo_nat_lang_3_yrs_other_p__9,demo_nat_lang_3_yrs_other_p_10,demo_nat_lang_3_yrs_other_p_11,demo_nat_lang_3_yrs_eng_p___0,demo_nat_lang_3_yrs_

In [11]:
data.duplicated(subset=['src_subject_id'])

0        False
1         True
2         True
3         True
4        False
         ...  
48802    False
48803     True
48804     True
48805     True
48806     True
Length: 48807, dtype: bool

In [12]:
adv.eventname.value_counts()

eventname
baseline_year_1_arm_1       11868
1_year_follow_up_y_arm_1    11220
2_year_follow_up_y_arm_1    10908
3_year_follow_up_y_arm_1    10123
4_year_follow_up_y_arm_1     4688
Name: count, dtype: int64

In [13]:
data.eventname.value_counts()

eventname
baseline_year_1_arm_1       11868
1_year_follow_up_y_arm_1    11220
2_year_follow_up_y_arm_1    10908
3_year_follow_up_y_arm_1    10123
4_year_follow_up_y_arm_1     4688
Name: count, dtype: int64

### Check

In [15]:
print(adv.shape)
adv.head()

(48807, 12)


,demo_prnt_ed_v2,demo_prtnr_ed_v2,src_subject_id,eventname,reshist_addr1_adi_unemp,reshist_addr1_adi_edu_l,reshist_addr1_adi_crowd,reshist_addr1_adi_pov,reshist_addr1_opat_kfrpp_avg,neighborhood1r_p,neighborhood2r_p,neighborhood3r_p
0,13.0,13.0,NDAR_INV003RTV85,baseline_year_1_arm_1,NaN,NaN,NaN,NaN,NaN,5.0,5.0,5.0
1,NaN,NaN,NDAR_INV003RTV85,1_year_follow_up_y_arm_1,NaN,NaN,NaN,NaN,NaN,5.0,5.0,4.0
2,NaN,NaN,NDAR_INV003RTV85,2_year_follow_up_y_arm_1,NaN,NaN,NaN,NaN,NaN,5.0,5.0,5.0
3,NaN,NaN,NDAR_INV003RTV85,3_year_follow_up_y_arm_1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,6.0,999.0,NDAR_INV005V6D2C,baseline_year_1_arm_1,9.991899,4.861931,2.440678,17.770597,0.424759,3.0,5.0,5.0


In [16]:
adv.eventname.value_counts()

eventname
baseline_year_1_arm_1       11868
1_year_follow_up_y_arm_1    11220
2_year_follow_up_y_arm_1    10908
3_year_follow_up_y_arm_1    10123
4_year_follow_up_y_arm_1     4688
Name: count, dtype: int64

#### Select onnly Wave 1

In [17]:
adv = adv[adv['eventname']=='baseline_year_1_arm_1']
print(adv.shape)

(11868, 12)


### rename variables

In [18]:
redict = {
    'reshist_addr1_adi_unemp': 'ADI1', 
    'reshist_addr1_adi_edu_l': 'ADI2', 
    'reshist_addr1_adi_crowd': 'ADI3', 
    'reshist_addr1_adi_pov': 'ADI4',
    'reshist_addr1_opat_kfrpp_avg': 'SocMob',
    'demo_prnt_ed_v2': 'peduh1',
    'demo_prtnr_ed_v2': 'peduh2',
    'neighborhood1r_p': 'neigh1',
    'neighborhood2r_p': 'neigh2',
    'neighborhood3r_p': 'neigh3',
    'src_subject_id': 'subID',
         }
adv.rename(columns=redict, inplace=True)

In [19]:
adv.head()

,peduh1,peduh2,subID,eventname,ADI1,ADI2,ADI3,ADI4,SocMob,neigh1,neigh2,neigh3
0,13.0,13.0,NDAR_INV003RTV85,baseline_year_1_arm_1,NaN,NaN,NaN,NaN,NaN,5.0,5.0,5.0
4,6.0,999.0,NDAR_INV005V6D2C,baseline_year_1_arm_1,9.991899,4.861931,2.440678,17.770597,0.424759,3.0,5.0,5.0
8,19.0,18.0,NDAR_INV007W6H7B,baseline_year_1_arm_1,6.254295,3.559711,2.930622,3.665339,0.552627,5.0,5.0,4.0
11,20.0,20.0,NDAR_INV00BD7VDC,baseline_year_1_arm_1,3.420132,0.635838,0.000000,1.204819,0.601058,5.0,5.0,5.0
14,15.0,NaN,NDAR_INV00CY2MDM,baseline_year_1_arm_1,7.476038,2.196885,2.690397,6.062932,0.529742,4.0,4.0,3.0


#### reverse code

In [20]:
# adv['peduh1'] = adv['peduh1']*-1
# adv['peduh2'] = adv['peduh2']*-1
# adv['SocMob'] = adv['SocMob']*-1

In [21]:
# adv.head()

#### Load & Merge full df 
* and merge adv with full

In [22]:
outpath = "/home/cglab/projects/abcd/rser/"
full = pd.read_csv(dpath + 'abcd5.1_rser_nback_5-30-24.csv', low_memory=False)

In [23]:
all = full.merge(adv, on='subID', how='left')
print(all.shape)
all.head()

(5754, 594)


,Aware,NoAcpt,Implse,Goals,Threat,subID,ThreatSQ,ThreatCB,eventname_yr2,rsfmri_var_cdk_insulalh_yr2,rsfmri_var_cdk_insularh_yr2,rsfmri_var_cdk_rlaclatelh_yr2,rsfmri_var_cdk_rlaclaterh_yr2,rsfmri_var_cdk_entorhinallh_yr2,rsfmri_var_cdk_entorhinalrh_yr2,rsfmri_var_scs_amygdalalh_yr2,rsfmri_var_scs_amygdalarh_yr2,rsfmri_var_scs_hpuslh_yr2,rsfmri_var_scs_hpusrh_yr2,rsfmri_var_scs_putamenlh_yr2,rsfmri_var_scs_putamenrh_yr2,rsfmri_var_scs_tplh_yr2,rsfmri_var_scs_tprh_yr2,rsfmri_var_scs_ventraldclh_yr2,rsfmri_var_scs_ventraldcrh_yr2,rsfmri_var_scs_aalh_yr2,rsfmri_var_scs_aarh_yr2,mrirsfd121_yr2,mrirsfd48_yr2,mrirsfd122_yr2,mrirsfd49_yr2,mrirsfd123_yr2,mrirsfd47_yr2,sa_scs_aalh_yr2,sa_scs_aarh_yr2,sa_scs_aglh_yr2,sa_scs_agrh_yr2,sa_scs_bs_yr2,sa_scs_cdelh_yr2,sa_scs_cderh_yr2,sa_scs_crcxlh_yr2,sa_scs_crcxrh_yr2,sa_scs_hplh_yr2,sa_scs_hprh_yr2,sa_scs_pllh_yr2,sa_scs_plrh_yr2,sa_scs_ptlh_yr2,sa_scs_ptrh_yr2,sa_scs_thplh_yr2,sa_scs_thprh_yr2,sa_scs_vtdclh_yr2,sa_scs_vtdcrh_yr2,df_scs_aalh_yr2,df_scs_aarh_yr2,df_scs_aglh_yr2,df_scs_agrh_yr2,df_scs_bs_yr2,df_scs_cdelh_yr2,df_scs_cderh_yr2,df_scs_crcxlh_yr2,df_scs_crcxrh_yr2,df_scs_hplh_yr2,df_scs_hprh_yr2,df_scs_pllh_yr2,df_scs_plrh_yr2,df_scs_ptlh_yr2,df_scs_ptrh_yr2,df_scs_thplh_yr2,df_scs_thprh_yr2,df_scs_vtdclh_yr2,df_scs_vtdcrh_yr2,sa_ngd_ad_yr2,sa_ngd_cgc_yr2,SaCParT5,sa_ngd_dt_yr2,sa_ngd_dla_yr2,sa_ngd_fo_yr2,sa_ngd_n_yr2,sa_ngd_rspltp_yr2,sa_ngd_sa_yr2,sa_ngd_smh_yr2,sa_ngd_smm_yr2,sa_ngd_vta_yr2,sa_ngd_vs_yr2,ders_aware_clar_score_3_year_yr2,ders_aware_clar_score_4_year_yr2,ders_awareness_score_3_year_yr2,ders_awareness_score_4_year_yr2,ders_goals_score_3_year_yr2,ders_goals_score_4_year_yr2,ders_impulse_score_3_year_yr2,ders_impulse_score_4_year_yr2,ders_nonaccept_score_3_year_yr2,ders_nonaccept_score_4_year_yr2,ders_total_score_3_year_yr2,ders_total_score_4_year_yr2,ders_aware_delta_4yr3_yr2,ders_awareness_delta_4yr3_yr2,ders_goals_delta_4yr3_yr2,ders_impulse_delta_4yr3_yr2,...,tfabwdp_1163_yr1,tnbasemdp_1051_yr1,tnbasemdp_1125_yr1,tnbasemdp_1089_yr1,tnbasemdp_1163_yr1,tfabwdp_1049_yr1,tfabwdp_1123_yr1,tnbasemdp_1049_yr1,tnbasemdp_1123_yr1,tfncr1bwdp_1049_yr1,tfabwdp_1048_yr1,tfabwdp_1122_yr1,tnbasemdp_1048_yr1,tnbasemdp_1122_yr1,tfabwdp_1063_yr1,tfabwdp_1137_yr1,tnbasemdp_1063_yr1,tnbasemdp_1137_yr1,tfabwdp_1066_yr1,tfabwdp_1140_yr1,tnbasemdp_1066_yr1,tnbasemdp_1140_yr1,tfmri_nback_all_757_yr1,tfmri_nback_all_791_yr1,tfmrinbackallsem_757_yr1,tfmrinbackallsem_791_yr1,tfmri_nback_all_772_yr1,tfmri_nback_all_806_yr1,tfmrinbackallsem_772_yr1,tfmrinbackallsem_806_yr1,tfmri_nback_all_777_yr1,tfmri_nback_all_811_yr1,tfmrinbackallsem_777_yr1,tfmrinbackallsem_811_yr1,tfmri_nback_all_765_yr1,tfmri_nback_all_799_yr1,tfmrinbackallsem_765_yr1,tfmrinbackallsem_799_yr1,tfmri_nback_all_772.1_yr1,tfmri_nback_all_806.1_yr1,tfmrinbackallsem_772.1_yr1,tfmrinbackallsem_806.1_yr1,tfmri_nback_all_774_yr1,tfmri_nback_all_808_yr1,tfmrinbackallsem_774_yr1,tfmrinbackallsem_808_yr1,tfmri_nback_all_771_yr1,tfmri_nback_all_805_yr1,tfmrinbackallsem_771_yr1,tfmrinbackallsem_805_yr1,tfmri_nback_all_773_yr1,tfmri_nback_all_807_yr1,tfmrinbackallsem_773_yr1,tfmrinbackallsem_807_yr1,tfmri_nback_all_759_yr1,tfmri_nback_all_793_yr1,tfmrinbackallsem_759_yr1,tfmrinbackallsem_793_yr1,tfmri_nback_all_780_yr1,tfmri_nback_all_814_yr1,tfmrinbackallsem_780_yr1,tfmrinbackallsem_814_yr1,tfmri_nback_all_763_yr1,tfmri_nback_all_797_yr1,tfmrinbackallsem_763_yr1,tfmrinbackallsem_797_yr1,tfmri_nback_all_752_yr1,tfmri_nback_all_786_yr1,tfmrinbackallsem_752_yr1,tfmrinbackallsem_786_yr1,tfmri_nback_all_770_yr1,tfmri_nback_all_804_yr1,tfmrinbackallsem_770_yr1,tfmrinbackallsem_804_yr1,tfmri_nback_all_751_yr1,tfmri_nback_all_785_yr1,tfmrinbackallsem_751_yr1,tfmrinbackallsem_785_yr1,tfmri_nback_all_761_yr1,tfmri_nback_all_795_yr1,tfmrinbackallsem_761_yr1,tfmrinbackallsem_795_yr1,rsfmri_meanmotion_yr1,Mot1nb,imgincl_nback_include_yr1,MotionNB,Rank_AmygL1,Rank_AmygL5,AmygRnkDif,peduh1,peduh2,eventname_y,ADI1,ADI2,ADI3,ADI4,SocMob,neigh1,ne

### Export

In [24]:
all.to_csv(outpath + 'abcd5.1_rser_nback_harsh_5-30-24.csv', index=False)